In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from collections import Counter
import math
import datetime as datetime
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
filename = 'data/fulldata_20160101_to_20191231.csv'


In [ ]:
data = pd.read_csv(filename)
data['datetime_str'] = data.loc[:,'TIME'].apply(datetime.datetime.utcfromtimestamp)

#data.columns

In [ ]:
df = pd.DataFrame(data=pd.concat([data['EN_PARA'] ,data['EN_ANTI']],ignore_index = True), columns = ['energy'])

df['pitch angle'] = pd.concat([data['PA_PARA'] ,data['PA_ANTI']],ignore_index = True)
df['flag'] = pd.concat([data['FLAG_PARA'] , -data['FLAG_ANTI']],ignore_index = True)
df['flux'] = pd.concat([data['FLUX_PARA'] ,data['FLUX_ANTI']],ignore_index = True)
df['eflux'] = pd.concat([data['EFLUX_PARA'] ,data['EFLUX_ANTI']],ignore_index = True)
df['xgsm'] = pd.concat([data['GSM_X'] ,data['GSM_X']],ignore_index = True)
df['ygsm'] = pd.concat([data['GSM_Y'] ,data['GSM_Y']],ignore_index = True)
df['zgsm'] = pd.concat([data['GSM_Z'] ,data['GSM_Z']],ignore_index = True)
df['beta'] = pd.concat([data['BETA'] ,data['BETA']],ignore_index = True)
df['imfBy'] = pd.concat([data['IMF_BY_PARA'] ,data['IMF_BY_ANTI']],ignore_index = True)
df['imfBz'] = pd.concat([data['IMF_BZ_PARA'] ,data['IMF_BZ_ANTI']],ignore_index = True)
df['storm_phase'] = pd.concat([data['STORM_PHASE'] ,data['STORM_PHASE']],ignore_index = True)
df['velocity_o_all'] = pd.concat([data['O_V'] ,data['O_V']],ignore_index = True)
df['density_o_all'] = pd.concat([data['O_N'] ,data['O_N']],ignore_index = True)
df['pressure_o_all'] = pd.concat([data['O_P'] ,data['O_P']],ignore_index = True)
df['density_h_all'] = pd.concat([data['H_N'] ,data['H_N']],ignore_index = True)
df['swp'] = pd.concat([data['SW_P_PARA'] ,data['SW_P_ANTI']],ignore_index = True)
df['kp'] = pd.concat([data['KP'] ,data['KP']],ignore_index = True)
df['dst'] = pd.concat([data['DST'] ,data['DST']],ignore_index = True)

df['kp_gt_2'] = df['kp'] > 2 
df['storm'] = df['storm_phase'] > 0
df['storm_phase'] = pd.Categorical(df['storm_phase']).rename_categories({0:'nonstorm',1:'prestorm',2:'main phase',3:'fast recovery', 4:'long recovery'})
df['region'] = df['beta']

df.loc[df['beta'] > 0 , 'region'] = 1
df.loc[df['beta'] > 0.5 , 'region'] = 2
df.loc[df['beta'] > 1 , 'region'] = 3


In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("Storm Phases", "Kp Index", "IMF Bz", "Solar wind Pressure"))

fig.add_trace(go.Histogram(x=df['storm_phase']), row=1, col=1)
fig.add_trace(go.Histogram(x=df['kp'], xbins=dict(start=0,end=7.0, size=0.1 )) ,row=1, col=2)
fig.add_trace(go.Histogram(x=df['imfBz']) ,row=2, col=1)
fig.add_trace(go.Histogram(x=df['swp']) ,row=2, col=2)

fig.update_xaxes(categoryorder='array', categoryarray= ['nonstorm','prestorm','main phase','fast recovery','long recovery'], row=1, col=1)

fig.update_layout(showlegend=False)


In [ ]:
dict(start=0.01,end=2000, size=1)

In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("Energy", "Flux", "Pitch Angle", "Region"))

fig.add_trace(go.Histogram(x=df['energy']), row=1, col=1)
fig.add_trace(go.Histogram(x=df['flux'], xbins=dict(start=0,end=200, size=1)) ,row=1, col=2)
fig.add_trace(go.Histogram(x=df['pitch angle']) ,row=2, col=1)
fig.add_trace(go.Histogram(x=df['region'], xbins=dict(start=0,end=4, size=1)) ,row=2, col=2)

#fig.update_xaxes(type="log", range = [0,5],row=1, col=1)
fig.update_xaxes(type="log", range = [0,2],row=1, col=2)
fig.update_xaxes(range = [1,4],row=2, col=2)

#fig.update_xaxes(type="log", row=1, col=2)
#fig.update_xaxes(type="log", row=2, col=2)

fig.update_layout(showlegend=False)


In [ ]:
go.Histogram(x=df['region'], xbins=dict(start=0,end=3, size=1))

In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("Vel_tot", "Density", "Pressure", "H+ density"))

fig.add_trace(go.Histogram(x=df['velocity_o_all']), row=1, col=1)
fig.add_trace(go.Histogram(x=df['density_o_all'], xbins=dict(start=0.01,end=10, size=0.1)) ,row=1, col=2)
fig.add_trace(go.Histogram(x=df['pressure_o_all'], xbins=dict(start=0.001,end=0.01, size=0.001)) ,row=2, col=1)
fig.add_trace(go.Histogram(x=df['density_h_all'], xbins=dict(start=0.01,end=10, size=0.1 )) ,row=2, col=2)

#fig.update_xaxes(type="log", range = [0,5],row=1, col=1)
fig.update_xaxes(type="log", range = [-1,1],row=1, col=2)
fig.update_xaxes(type="log", range = [-4,-2],row=2, col=1)
fig.update_xaxes(type="log", range = [-1,1],row=2, col=2)

#fig.update_xaxes(type="log", row=1, col=2)
#fig.update_xaxes(type="log", row=2, col=2)

fig.update_layout(showlegend=False)


In [ ]:
index =  df['flux'].notnull()
df['abs_flag'] = df['flag'].apply(abs)
fig = px.density_heatmap(df, x = 'xgsm', y = 'zgsm', marginal_x="histogram", marginal_y="histogram"
                        , z ='abs_flag', histfunc ='avg', nbinsx=20, nbinsy=16
                         , range_x=[-30,10],range_y=[-12,12]
                         ,facet_row="storm", facet_col="kp_gt_2", title='Occurrence Frequency')
fig.show()

In [ ]:
index =  df['flux'].notnull()
df['abs_flag'] = df['flag'].apply(abs)
fig = px.density_heatmap(df, x = 'xgsm', y = 'zgsm', marginal_x="histogram", marginal_y="histogram"
                         , z="flux", histfunc="avg", nbinsx=20, nbinsy=16
                         , range_x=[-30,10],range_y=[-12,12]
                         ,facet_row="storm", facet_col="kp_gt_2", title='flux' )
fig.show()